In [1]:
import posix_ipc
import mmap
import numpy as np
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
shm = posix_ipc.SharedMemory(name='/shm_20231229_order')
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_READ)
shm_mmap.close()
shm.close_fd()
shm.unlink()  

In [21]:
data_path = '/home/haolinl/converted_parquet/20231229_order.parquet'
# Load data as a Pandas DataFrame
df = pd.read_parquet(data_path)

# remove last column
# df = df.iloc[:, :-1]
df.dtypes

OrderTime      float64
sysid          float64
OrderPrice     float64
OrderVolume    float64
BSFlag         float64
OrderType      float64
stock_code     float64
dtype: object

In [12]:
df['stock_code'] = df['stock_code'].astype('S20')


In [13]:
df.dtypes

OrderTime      float64
sysid          float64
OrderPrice     float64
OrderVolume    float64
BSFlag         float64
OrderType      float64
stock_code        |S20
dtype: object

In [22]:
# OrderTime      float64
# sysid          float64
# OrderPrice     float64
# OrderVolume    float64
# BSFlag         float64
# OrderType      float64
# stock_code        |S20
# dtype: object
# Convert to NumPy array for sharing
array = df.to_numpy()

In [17]:
array = array.astype('S20')

In [23]:

# Create shared memory
shm_name = f"/shm_20231229_order"  # Shared memory name must start with a slash
shm = posix_ipc.SharedMemory(
    name=shm_name,
    flags= posix_ipc.O_CREAT | posix_ipc.O_EXCL,
    mode=0o600,
    size=array.nbytes
)
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_WRITE)
# Create a NumPy array from the mapped memory
shm_arr = np.ndarray(array.shape, dtype=array.dtype, buffer=shm_mmap)
shm_arr[:] = array[:]


In [24]:
print(shm.name, shm.size, shm_arr.shape, shm_arr.dtype)

/shm_20231229_order 7737071384 (138161989, 7) float64


In [25]:
shm = posix_ipc.SharedMemory(name='/shm_20231229_order')
# Map the shared memory into the process's address space
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_READ)
# Create a NumPy array from the mapped memory
shm_arr = np.ndarray((138161989, 7), dtype='float64', buffer=shm_mmap)
df = pd.DataFrame(shm_arr)
df

,0,1,2,3,4,5,6
0,1.703813e+09,248.0,23.30,3600.0,1.0,4.0,601699.0
1,1.703813e+09,818.0,22.50,400.0,1.0,4.0,601699.0
2,1.703813e+09,1103.0,23.84,100.0,1.0,4.0,601699.0
3,1.703813e+09,2716.0,21.97,500.0,1.0,4.0,601699.0
4,1.703813e+09,3324.0,22.29,3000.0,1.0,4.0,601699.0
...,...,...,...,...,...,...,...
138161984,1.703833e+09,33482493.0,6.99,2400.0,0.0,2.0,2451.0
138161985,1.703833e+09,33482521.0,7.00,1200.0,0.0,2.0,2451.0
138161986,1.703833e+09,33482902.0,6.99,7800.0,0.0,2.0,2451.0
138161987,1.703833e+09,33482956.0,7.00,1500.0,0.0,2.0,2451.0


In [18]:
shm1 = posix_ipc.SharedMemory(name='/shm_20231226_order')
# Map the shared memory into the process's address space
shm_mmap1 = mmap.mmap(shm1.fd, shm1.size, access=mmap.ACCESS_READ)
# Create a NumPy array from the mapped memory
shm_arr1 = np.ndarray((118490170, 7), dtype='object', buffer=shm_mmap1)
df1 = pd.DataFrame(shm_arr1)
df1

,0,1,2,3,4,5,6
0,1703553300.05,188.0,25.25,500.0,1.0,4.0,sh601699
1,1703553300.15,639.0,24.1,2000.0,1.0,4.0,sh601699
2,1703553300.15,807.0,25.38,100.0,1.0,4.0,sh601699
3,1703553300.29,1100.0,23.07,200.0,1.0,4.0,sh601699
4,1703553300.31,1169.0,23.88,500.0,1.0,4.0,sh601699
...,...,...,...,...,...,...,...
118490165,1703573999.78,28934288.0,6.62,100.0,0.0,2.0,sz002451
118490166,1703573999.83,28934584.0,6.59,14200.0,1.0,2.0,sz002451
118490167,1703573999.89,28934895.0,6.62,100.0,0.0,2.0,sz002451
118490168,1703573999.9,28934974.0,6.62,100.0,0.0,2.0,sz002451
